# 01 — Seed reference data (Unity Catalog + Delta) [web:9][web:15]

Creates small reference tables under the `ref` schema used by subsequent notebooks.


In [ ]:
%pip install faker==25.2.0


In [ ]:
# Configuration (Databricks widgets)
# These widgets make the demo portable across workspaces/accounts.
# If you're running this outside a Databricks notebook, it will fall back to defaults.

DEFAULT_CATALOG = "rtpa_catalog"
DEFAULT_SCHEMA_REF = "credentialing_ref"
DEFAULT_SCHEMA_BRONZE = "credentialing_bronze"
DEFAULT_SCHEMA_SILVER = "credentialing_silver"
DEFAULT_SCHEMA_GOLD = "credentialing_gold"

DEFAULT_N_PROVIDERS = 200
DEFAULT_DAYS_SCHEDULE = 14
DEFAULT_SEED = 42

try:
    dbutils.widgets.text("catalog", DEFAULT_CATALOG, "Catalog")
    dbutils.widgets.text("schema_ref", DEFAULT_SCHEMA_REF, "Schema (ref)")
    dbutils.widgets.text("schema_bronze", DEFAULT_SCHEMA_BRONZE, "Schema (bronze)")
    dbutils.widgets.text("schema_silver", DEFAULT_SCHEMA_SILVER, "Schema (silver)")
    dbutils.widgets.text("schema_gold", DEFAULT_SCHEMA_GOLD, "Schema (gold)")

    dbutils.widgets.text("n_providers", str(DEFAULT_N_PROVIDERS), "N providers")
    dbutils.widgets.text("days_schedule", str(DEFAULT_DAYS_SCHEDULE), "Days schedule")
    dbutils.widgets.text("seed", str(DEFAULT_SEED), "Random seed")

    catalog = dbutils.widgets.get("catalog") or DEFAULT_CATALOG
    schema_ref = dbutils.widgets.get("schema_ref") or DEFAULT_SCHEMA_REF
    schema_bronze = dbutils.widgets.get("schema_bronze") or DEFAULT_SCHEMA_BRONZE
    schema_silver = dbutils.widgets.get("schema_silver") or DEFAULT_SCHEMA_SILVER
    schema_gold = dbutils.widgets.get("schema_gold") or DEFAULT_SCHEMA_GOLD

    N_PROVIDERS = int(dbutils.widgets.get("n_providers") or DEFAULT_N_PROVIDERS)
    DAYS_SCHEDULE = int(dbutils.widgets.get("days_schedule") or DEFAULT_DAYS_SCHEDULE)
    SEED = int(dbutils.widgets.get("seed") or DEFAULT_SEED)
except Exception:
    catalog = DEFAULT_CATALOG
    schema_ref = DEFAULT_SCHEMA_REF
    schema_bronze = DEFAULT_SCHEMA_BRONZE
    schema_silver = DEFAULT_SCHEMA_SILVER
    schema_gold = DEFAULT_SCHEMA_GOLD

    N_PROVIDERS = DEFAULT_N_PROVIDERS
    DAYS_SCHEDULE = DEFAULT_DAYS_SCHEDULE
    SEED = DEFAULT_SEED

# Derived helpers
fq = lambda sch, tbl: f"{catalog}.{sch}.{tbl}"


In [ ]:
# Unity Catalog bootstrap (you may need permissions to create catalogs/schemas)
spark.sql(f"CREATE CATALOG IF NOT EXISTS {catalog}")
spark.sql(f"USE CATALOG {catalog}")
for sch in [schema_ref, schema_bronze, schema_silver, schema_gold]:
    spark.sql(f"CREATE SCHEMA IF NOT EXISTS {catalog}.{sch}")


## Create reference tables
Tables created: `ref.facility`, `ref.department`, `ref.procedure`, `ref.credential_type`, `ref.payer`.


In [ ]:
from pyspark.sql.types import *

# ref.facility
facility_rows = [
    ("FAC-001", "Manhattan General", "NYC"),
    ("FAC-002", "Brooklyn Community", "NYC"),
    ("FAC-003", "Queens Regional", "NYC"),
    ("FAC-004", "Bronx Medical Center", "NYC"),
]
facility_schema = StructType([
    StructField("facility_id", StringType(), False),
    StructField("facility_name", StringType(), False),
    StructField("region", StringType(), False),
])
facility_df = spark.createDataFrame(facility_rows, facility_schema)

# ref.department
department_rows = [
    ("D-001", "FAC-001", "Emergency"),
    ("D-002", "FAC-001", "Surgery"),
    ("D-003", "FAC-002", "Emergency"),
    ("D-004", "FAC-002", "Anesthesiology"),
    ("D-005", "FAC-003", "ICU"),
    ("D-006", "FAC-004", "Cardiology"),
]
department_schema = StructType([
    StructField("dept_id", StringType(), False),
    StructField("facility_id", StringType(), False),
    StructField("dept_name", StringType(), False),
])
department_df = spark.createDataFrame(department_rows, department_schema)

# ref.procedure
procedure_rows = [
    ("PROC-ER-INT",   "ER Initial Triage",        "D-001", False, True),
    ("PROC-ER-SEDS",  "Moderate Sedation",        "D-001", True,  True),
    ("PROC-SURG-APP", "Appendectomy",             "D-002", True,  False),
    ("PROC-ANES-GEN", "General Anesthesia",        "D-004", True,  True),
    ("PROC-ICU-VNT",  "Ventilator Management",     "D-005", True,  True),
    ("PROC-CARD-ECG", "ECG Interpretation",        "D-006", False, False),
]
procedure_schema = StructType([
    StructField("procedure_code", StringType(), False),
    StructField("procedure_name", StringType(), False),
    StructField("dept_id", StringType(), False),
    StructField("requires_privilege", BooleanType(), False),
    StructField("requires_acls", BooleanType(), False),
])
procedure_df = spark.createDataFrame(procedure_rows, procedure_schema)

# ref.credential_type
credential_type_rows = [
    ("STATE_MED_LICENSE", "State medical license (required for practice)", 365*2),
    ("ACLS",              "Advanced Cardiovascular Life Support",         365*2),
    ("DEA",               "Controlled substance registration (DEA)",      365*3),
]
credential_type_schema = StructType([
    StructField("cred_type", StringType(), False),
    StructField("cred_desc", StringType(), False),
    StructField("renewal_cycle_days", IntegerType(), False),
])
credential_type_df = spark.createDataFrame(credential_type_rows, credential_type_schema)

# ref.payer
payer_rows = [
    ("PAY-001", "Medicare"),
    ("PAY-002", "Medicaid"),
    ("PAY-003", "Aetna"),
    ("PAY-004", "UnitedHealthcare"),
    ("PAY-005", "Blue Cross Blue Shield"),
]
payer_schema = StructType([
    StructField("payer_id", StringType(), False),
    StructField("payer_name", StringType(), False),
])
payer_df = spark.createDataFrame(payer_rows, payer_schema)

# Write as Delta tables in Unity Catalog
facility_df.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable(fq(schema_ref, "facility"))
department_df.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable(fq(schema_ref, "department"))
procedure_df.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable(fq(schema_ref, "procedure"))
credential_type_df.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable(fq(schema_ref, "credential_type"))
payer_df.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable(fq(schema_ref, "payer"))


## Validate
Count each table and display at least one.


In [ ]:
tables = ["facility", "department", "procedure", "credential_type", "payer"]
for t in tables:
    c = spark.read.table(fq(schema_ref, t)).count()
    print(f"{fq(schema_ref, t)}: {c:,}")

display(spark.read.table(fq(schema_ref, "procedure")).orderBy("procedure_code"))
